In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 
!pip install krovetzstemmer
import krovetzstemmer
import nltk
nltk.download("popular")
from nltk.corpus import stopwords
from krovetzstemmer import Stemmer
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

!pip install -q keras
import keras
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import Sequential
from keras import layers


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

Using TensorFlow backend.


cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X_train=pd.read_csv("/content/drive/My Drive/X_TRAIN-QUORA.csv",index_col="Unnamed: 0")
X_test=pd.read_csv("/content/drive/My Drive/X_TEST-QUORA.csv",index_col="Unnamed: 0")
y_train=pd.read_csv("/content/drive/My Drive/Y_TRAIN-QUORA.csv",header=None,index_col=0)
y_test=pd.read_csv("/content/drive/My Drive/Y_TEST-QUORA.csv",header=None,index_col=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(353751, 14)
(50536, 14)
(353751, 1)
(50536, 1)


In [ ]:
"""
Current Features:

#of common unigram (wordshare)
shared-tfidf
word mover distance
q1 freq
q2 freq
"""

'\nCurrent Features:\n\n#of common unigram (wordshare)\nshared-tfidf\nword mover distance\nq1 freq\nq2 freq\n'

In [ ]:
len(np.array(X_train))

353751

#New Features
<br>
<br>

Lengths & Difference of lengths & Jaccard Similarity 

The Jaccard Index, also known as the Jaccard similarity coefficient, is a statistic used in understanding the similarities between sample sets. The measurement emphasizes similarity between finite sample sets, and is formally defined as the size of the intersection divided by the size of the union of the sample sets.

In [ ]:
lengthq1_train=[]
lengthq2_train=[]
lengthq1_test=[]
lengthq2_test=[]
jaccard_sim_train=[]
jaccard_sim_test=[]
len_dif_train=[]
len_dif_test=[]
for i in range(0,len(X_train)):
  len1=len(str(X_train.iloc[i,2]).split(" "))
  len2=len(str(X_train.iloc[i,3]).split(" "))
  wordshare=(X_train.iloc[i,8])

  len_dif=abs(len1-len2)
  len_dif_train.append(len_dif)

  lengthq1_train.append(len1)
  lengthq2_train.append(len2)
  jaccard_sim_train.append(wordshare/ (len1+len2-wordshare))

for i in range(0,len(X_test)):
  len1=len(str(X_test.iloc[i,2]).split(" "))
  len2=len(str(X_test.iloc[i,3]).split(" "))
  wordshare=(X_test.iloc[i,8])


  len_dif=abs(len1-len2)
  len_dif_test.append(len_dif)

  lengthq1_test.append(len1)
  lengthq2_test.append(len2)
  jaccard_sim_test.append(wordshare/ (len1+len2-wordshare))

X_train['len_q1']=lengthq1_train
X_train['len_q2']=lengthq2_train

X_test['len_q1']=lengthq1_test
X_test['len_q2']=lengthq1_test

X_train['jaccard_sim']=jaccard_sim_train
X_test['jaccard_sim']=jaccard_sim_test

X_train['len_dif']=len_dif_train
X_test['len_dif']=len_dif_test


Cosine Distance

<br>
<br>

In [ ]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

text1 = 'This is a foo bar sentence .'
text2 = 'This sentence is similar to a foo bar sentence .'

vector1 = text_to_vector("anil")
vector2 = text_to_vector("anil-se")

cosine = get_cosine(vector1, vector2)

print ('Cosine:', cosine)

Cosine: 0.7071067811865475


In [ ]:
cos_train=[]
cos_test=[]
for i in range (0,len(X_train)):
  s1=str(X_train.iloc[i,2])
  s2=str(X_train.iloc[i,3])
  s1 = text_to_vector(s1)
  s2 = text_to_vector(s2)
  cos=get_cosine(s1, s2)
  cos_train.append(cos)
for i in range (0,len(X_test)):
  s1=str(X_test.iloc[i,2])
  s2=str(X_test.iloc[i,3])
  s1 = text_to_vector(s1)
  s2 = text_to_vector(s2)
  cos=get_cosine(s1, s2)
  cos_test.append(cos)
X_train["cosine_distance"]=cos_train
X_test["cosine_distance"]=cos_test

In [ ]:
X_train

,qid1,qid2,question1,question2,question1_tokenized,question2_tokenized,question1-stemmed,question2-stemmed,wordShare,common_words,shared-tfidf,WMD,Q1-FREQ,Q2-FREQ,len_q1,len_q2,jaccard_sim,len_dif,cosine_distance
290384,411670,411671,study excellent financial analyst,financial analyst use sql,"['study', 'excellent', 'financial', 'analyst']","['financial', 'analysts', 'use', 'sql']","['study', 'excellent', 'financial', 'analyst']","['financial', 'analyst', 'use', 'sql']",2,analyst financial,0.000141,0.710545,1,1,4,4,0.333333,0,0.500000
184897,282382,282383,example subsistence farming,example simple farming tool,"['examples', 'subsistence', 'farming']","['examples', 'simple', 'farming', 'tools']","['example', 'subsistence', 'farming']","['example', 'simple', 'farming', 'tool']",2,example farming,0.000137,0.783249,1,1,3,4,0.400000,1,0.577350
107350,176640,176641,parent wo let choose future job college follow...,pass go appear jee pls suggest best cs college...,"['parents', 'wo', 'let', 'choose', 'future', '...","['passed', 'going', 'appear', 'jee', 'pls', 's...","['parent', 'wo', 'let', 'choose', 'future', 'j...","['pass', 'go', 'appear', 'jee', 'pls', 'sugges...",1,college,0.000100,1.007334,1,1,10,14,0.043478,4,0.084515
279787,399275,250080,internal parts computer purpose,internal parts computer system purpose serve,"['internal', 'parts', 'computer', 'purpose']","['internal', 'parts', 'computer', 'system', 'p...","['internal', 'parts', 'computer', 'purpose']","['internal', 'parts', 'computer', 'system', 'p...",4,purpose internal parts computer,0.000198,0.545631,1,1,4,6,0.666667,2,0.816497
58238,102205,102206,delete amazon account,delete amazon account,"['delete', 'amazon', 'account']","['delete', 'amazon', 'account']","['delete', 'amazon', 'account']","['delete', 'amazon', 'account']",3,delete account amazon,0.000172,0.000000,1,1,3,3,1.000000,0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259180,374920,374921,power positive thinking,cultivate power positive thinking,"['power', 'positive', 'thinking']","['cultivate', 'power', 'positive', 'thinking']","['power', 'positive', 'thinking']","['cultivate', 'power', 'positive', 'thinking']",3,positive power thinking,0.000173,0.506248,1,1,3,4,0.750000,1,0.866025
365841,495977,17978,things new employee know go first day tennant,things new employee know go first day,"['things', 'new', 'employees', 'know', 'going'...","['things', 'new', 'employees', 'know', 'going'...","['things', 'new', 'employee', 'know', 'go', 'f...","['things', 'new', 'employee', 'know', 'go', 'f...",7,day know new employee first go things,0.000264,0.311377,1,59,8,7,0.875000,1,0.935414
131933,9848,127922,current winning presidential election,bias aside point time think win presidential e...,"['currently', 'winning', 'presidential', 'elec...","['biases', 'aside', 'point', 'time', 'think', ...","['current', 'winning', 'presidential', 'electi...","['bias', 'aside', 'point', 'time', 'think', 'w...",2,presidential election,0.000141,0.710542,9,13,4,8,0.200000,4,0.353553
146868,231910,231911,movies leading actor become mafia,fix internet connection available problem mobi...,"['movies', 'leading', 'actor', 'becomes', 'maf...","['fix', 'internet', 'connection', 'available',...","['movies', 'leading', 'actor', 'become', 'mafia']","['fix', 'internet', 'connection', 'available',...",0,NaN,0.000000,0.892477,1,1,5,9,0.000000,4,0.000000


Final version of Xtrain and Xtest

In [ ]:
X_train.columns

Index(['qid1', 'qid2', 'question1', 'question2', 'question1_tokenized',
       'question2_tokenized', 'question1-stemmed', 'question2-stemmed',
       'wordShare', 'common_words', 'shared-tfidf', 'WMD', 'Q1-FREQ',
       'Q2-FREQ', 'len_q1', 'len_q2', 'jaccard_sim', 'len_dif',
       'cosine_distance'],
      dtype='object')

In [ ]:
X_train=X_train[["wordShare","shared-tfidf","WMD","Q1-FREQ","Q2-FREQ","len_q1","len_q2","jaccard_sim","cosine_distance","len_dif"]]
X_test=X_test[["wordShare","shared-tfidf","WMD","Q1-FREQ","Q2-FREQ","len_q1","len_q2","jaccard_sim","cosine_distance","len_dif"]]

#Normalization

We only have to normalize shared-weight since maximum number is very less we are multiply it by 1000 to get it on 0-1 scale.

In [ ]:
max(X_train["shared-tfidf"])

0.0004067850397186828

In [ ]:
X_train['shared-tfidf'] = X_train['shared-tfidf'].apply(lambda x: x*1000)
X_test['shared-tfidf'] = X_test['shared-tfidf'].apply(lambda x: x*1000)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#Now ready.

#Models

#Logistic Regression

<br>
<br>
<br>

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

model_logistic = LogisticRegression()

# Create regularization penalty space
pen = ['l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 3)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=pen)


# Create grid search using 5-fold cross validation
clf = RandomizedSearchCV(model_logistic, hyperparameters, cv=5, verbose=1)

# Fit grid search
best_model = clf.fit(X_train, y_train)


# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])





/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:43

Best Penalty: l2
Best C: 10000.0


In [ ]:
#USE BEST MODEL TO TRAIN ALL DATA.
# Predict target vector
prediction=best_model.predict(X_test)

a=classification_report(y_test, prediction)
b=confusion_matrix(y_test, prediction)

print(a)
print(b)


              precision    recall  f1-score   support

           0       0.69      0.89      0.78     31784
           1       0.64      0.32      0.43     18752

    accuracy                           0.68     50536
   macro avg       0.66      0.61      0.60     50536
weighted avg       0.67      0.68      0.65     50536

[[28320  3464]
 [12666  6086]]


Logistic Train Error

In [ ]:
#USE BEST MODEL TO TRAIN ALL DATA.
# Predict target vector
prediction=best_model.predict(X_train)

a=classification_report(y_train, prediction)
b=confusion_matrix(y_train, prediction)

print(a)
print(b)

              precision    recall  f1-score   support

           0       0.76      0.85      0.80    223240
           1       0.68      0.55      0.61    130511

    accuracy                           0.74    353751
   macro avg       0.72      0.70      0.71    353751
weighted avg       0.73      0.74      0.73    353751

[[188918  34322]
 [ 58150  72361]]


#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42, verbose =1)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())


import pprint
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = n_estimators = [10,20,30,40,50,60,80,100]

# Number of features to consider at every split
# Maximum number of levels in tree
max_depth = [4,6,8,10]


random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth}
rf=RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
best_rf_model=rf_random.fit(X_train, y_train)


#best params -> depth None est-> 400

# View best hyperparameters
print('Best Penalty:', best_rf_model.best_estimator_.get_params()['n_estimators'])
print('Best C:', best_rf_model.best_estimator_.get_params()['max_depth'])


Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 1,
 'warm_start': False}
Fitting 3 folds for each of 32 candidates, totalling 96 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 32 is smaller than n_iter=100. Running 32 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 15.4min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best Penalty: 30
Best C: 10


In [ ]:
#USE BEST MODEL of random forest predict test data
# Predict target vector
prediction=best_rf_model.predict(X_test)

a=classification_report(y_test, prediction)
b=confusion_matrix(y_test, prediction)

print(a)
print(b)


              precision    recall  f1-score   support

           0       0.70      0.94      0.80     31784
           1       0.76      0.32      0.45     18752

    accuracy                           0.71     50536
   macro avg       0.73      0.63      0.63     50536
weighted avg       0.72      0.71      0.67     50536

[[29815  1969]
 [12666  6086]]


In [ ]:
#Train error

#USE BEST MODEL of random forest predict test data
# Predict target vector
prediction=best_rf_model.predict(X_train)

a=classification_report(y_train, prediction)
b=confusion_matrix(y_train, prediction)

print(a)
print(b)

              precision    recall  f1-score   support

           0       0.81      0.90      0.85    223240
           1       0.79      0.63      0.70    130511

    accuracy                           0.80    353751
   macro avg       0.80      0.76      0.77    353751
weighted avg       0.80      0.80      0.79    353751

[[201011  22229]
 [ 48539  81972]]


In [ ]:
import pickle
pickle.dump(best_rf_model, open("RF-MODEL-QUORA", 'wb')) # Save RF model
pickle.dump(best_model, open("LR-MODEL-QUORA", 'wb')) # save Logistic reg model

"""
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)
"""

"\n# load the model from disk\nloaded_model = pickle.load(open(filename, 'rb'))\nresult = loaded_model.score(X_test, Y_test)\nprint(result)\n"

#XGBOOST
<br>
<br>
<br>

In [ ]:
import xgboost as xgb


xgb_model=xgb.XGBClassifier(learning_rate= 0.01, max_depth = 60, n_estimators= 100)

xgb_model.fit(X_train, y_train) 

prediction_xgb=xgb_model.predict(X_test)
a=classification_report(y_test, prediction_xgb)
b=confusion_matrix(y_test, prediction_xgb)
print(a,b)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.72      0.89      0.79     31784
           1       0.68      0.41      0.51     18752

    accuracy                           0.71     50536
   macro avg       0.70      0.65      0.65     50536
weighted avg       0.70      0.71      0.69     50536
 [[28152  3632]
 [11080  7672]]


In [ ]:
import xgboost as xgb


xgb_model=xgb.XGBClassifier(learning_rate= 0.001, max_depth = 60, n_estimators= 100)

xgb_model.fit(X_train, y_train) 

prediction_xgb=xgb_model.predict(X_test)
a=classification_report(y_test, prediction_xgb)
b=confusion_matrix(y_test, prediction_xgb)
print(a,b)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.72      0.87      0.79     31784
           1       0.65      0.41      0.51     18752

    accuracy                           0.70     50536
   macro avg       0.68      0.64      0.65     50536
weighted avg       0.69      0.70      0.68     50536
 [[27667  4117]
 [10985  7767]]


In [ ]:
#Train Error
prediction_xgb=xgb_model.predict(X_train)
a=classification_report(y_train, prediction_xgb)
b=confusion_matrix(y_train, prediction_xgb)
print(a,b)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91    223240
           1       0.87      0.81      0.84    130511

    accuracy                           0.89    353751
   macro avg       0.88      0.87      0.88    353751
weighted avg       0.89      0.89      0.89    353751
 [[207565  15675]
 [ 24388 106123]]
